# Optimisation des warehouses

## Import des modules et des données

In [1]:
from sklearn.cluster import KMeans
import folium
import pandas as pd
import branca.colormap as cm
import numpy as np
from edc import utils
from edc import maps
from edc import processing

%load_ext autoreload
%autoreload 2

In [2]:
df_orders_lines = pd.read_csv('..\data\intermediate_data\df_orders_lines.csv')
df_carte_point = pd.read_csv('..\data\intermediate_data\df_carte_point.csv')

df_cities = pd.read_csv("../data/cities.csv")
df_routes = pd.read_csv("../data/routes_v2.csv")
df_orders = pd.read_csv("../data/orders.csv")

CO2_per_km = 0.795

## Calcul des nouveaux warehouses

On utilise un kmeans pour segmenter le territoire en 5 en fonction des distances entre les villes et les délais de livraison. On récupère les centroïdes de ces clusters.

In [3]:
kmeans = KMeans(n_clusters=5)
kmeans.fit(df_orders_lines[['lat_delivery','lng_delivery']],df_orders_lines.order_total_volume)
new_warehouse = pd.DataFrame(kmeans.cluster_centers_,columns=['lat','lng'])

In [4]:
df_orders_lines['new_warehouse'] = kmeans.labels_
df_orders_lines.rename(columns={'lat_delivery':'lat' , 'lng_delivery':'lng'},inplace=True)
new_warehouse['index_warehouse'] = new_warehouse.index

En noir les entrepôts actuels, en couleur les entrepôts optimaux.

In [5]:
m = folium.Map()

colormap = cm.LinearColormap(colors=['white','black'], vmin=df_carte_point.order_total_volume.min(),vmax=df_carte_point.order_total_volume.max())
colormap.caption = 'order total volume'

df_orders_lines.apply(lambda point : maps.plotDot(m,point,color=maps.colors[point.new_warehouse]), axis = 1)
m.add_child(colormap,name='order total volume')

#Set the zoom to the maximum possible
m.fit_bounds(m.get_bounds())

df_cities[df_cities.is_warehouse].apply(lambda point : maps.plotMarker(m,point,color='black'), axis = 1)
new_warehouse.apply(lambda point : maps.plotMarker(m,point,color = maps.colors[int(point.index_warehouse)],with_tooltip=None), axis = 1)

m

In [6]:
df_routes["stops_list"] = df_routes.stops.apply(lambda row : row.split(' > '))
df_routes.drop(columns=['stops'],inplace=True)

In [7]:
n_trajets = 100

## Scénario 0

On fait rien

In [8]:
int(df_routes.sample(n_trajets,random_state=42).total_distance.sum()* CO2_per_km)

32606

In [9]:
from edc import optimisation
df_routes_optimized_0_small = df_routes.sample(n_trajets,random_state=42)
df_routes_optimized_0_small['stops_list_ordonnee_f'] = df_routes_optimized_0_small.apply(lambda row :optimisation.optimiser_trajectoire(utils.remove_consecutive_duplicate([utils.donnees_cities_with_name(row.from_warehouse)]+[utils.donnees_cities_with_name(city) for city in row.stops_list])),axis=1)
df_routes_optimized_0_small['total_distance_f'] = df_routes_optimized_0_small.apply(lambda row : utils.total_distance_trip([row.from_warehouse]+row.stops_list_ordonnee_f), axis=1)

In [10]:
int(df_routes_optimized_0_small.total_distance_f.sum()* CO2_per_km)

47870

In [11]:
df_cities_0 = pd.read_csv("../data/cities.csv")
df_warehouses_0 = pd.read_csv("../data/warehouses.csv")

In [12]:
liste_warehouses_0 = ['Cergy','Reims','Avignon','Montauban','Clermont-Ferrand']
df_orders

,order_id,package_id,package_volume,delivery_location,from_warehouse,order_date,delivered_date,n_units,order_total_volume
0,EtPmaD3APj8aRNeSC3Yx7R,Z8722BWG8WfyrU3jeWsWGo,0.2732,Nice,Avignon,2021-01-01,2021-01-04 00:00:00,27.0,7.3764
1,4X4RBUwTXGBMVWdEj8Gti4,4oxrPXkner2kvXqvLQiCXF,0.2384,Issy-les-Moulineaux,Cergy,2021-01-01,2021-01-03 00:00:00,41.0,9.7744
2,GDhuTcBnr9xdRhFb2jLYcW,mnEb7xGXa9dFMLPAkrZZoh,1.4152,Toulouse,Montauban,2021-01-01,2021-01-08 00:00:00,35.0,49.5320
3,3EZDvpzw5DdxXyJ66Py75J,3nwbxBUemtMG75cVPs7q7R,1.3451,Poitiers,Montauban,2021-01-01,2021-01-09 00:00:00,21.0,28.2471
4,Fpvw9NfnexSxjzLvyPWKbQ,mnEb7xGXa9dFMLPAkrZZoh,1.4152,Épinal,Reims,2021-01-01,2021-01-09 00:00:00,22.0,31.1344
...,...,...,...,...,...,...,...,...,...
37327,8QVSak34zC2xgTpHSirnei,Z8722BWG8WfyrU3jeWsWGo,0.2732,Colombes,Cergy,2021-06-29,NaN,10.0,2.7320
37328,3JuWiSMrsXLnh8UzR34uzr,bfvaknKja6jMJYp56vtjgf,0.0072,Lille,Reims,2021-06-29,NaN,22.0,0.1584
37329,VaqbTgtahJZEcrUnSepiGG,KmBprCffsWobByaWonfSXN,0.0485,Montpellier,Avignon,2021-06-29,NaN,41.0,1.9885
37330,4tzaEQGmAUmvMiEpcBnTfB,3CCHeWCD6EnM24iAxbCfCr,1.5890,Dunkerque,Cergy,2021-06-29,NaN,18.0,28.6020


In [13]:
import numpy as np

In [14]:
df_cities_0["warehouse"] = df_cities_0.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_0))
df_cities_0["index_warehouse"] = df_cities_0.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_0,return_index=True))
df_warehouses_0 = df_cities[df_cities.city.isin(liste_warehouses_0)]
df_warehouses_0.index = np.arange(0,len(df_warehouses_0))
df_warehouses_0['index_warehouse'] = df_warehouses_0.index.copy()

d:\Documents\Cours\3A\M2AD\Novembre\edc_ekimetrics\venv\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [15]:
df_warehouses_0

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper,is_warehouse,index_warehouse
0,Reims,49.2628,4.0347,France,FR,Grand Est,minor,182460.0,182211.0,True,0
1,Clermont-Ferrand,45.7831,3.0824,France,FR,Auvergne-Rhône-Alpes,minor,143886.0,143886.0,True,1
2,Avignon,43.9500,4.8075,France,FR,Provence-Alpes-Côte d’Azur,minor,93671.0,91729.0,True,2
3,Cergy,49.0361,2.0631,France,FR,Île-de-France,minor,65177.0,65177.0,True,3
4,Montauban,44.0181,1.3558,France,FR,Occitanie,minor,60810.0,60810.0,True,4


In [16]:
m = folium.Map()

df_cities_0.apply(lambda point : maps.plotDot(m,point,color=maps.colors[point.index_warehouse],with_tooltip=False), axis = 1)

#Set the zoom to the maximum possible
m.fit_bounds(m.get_bounds())

df_warehouses_0.apply(lambda point : maps.plotMarker(m,point,color = maps.colors[int(point.index_warehouse)],with_tooltip=False), axis = 1)

m

## Scénario 1

On supprime l'entrepôt de Clermont-Ferrant

In [17]:
df_cities_1 = pd.read_csv("../data/cities.csv")
liste_warehouses_1 = ['Cergy','Reims','Avignon','Montauban']
df_warehouse_1 = df_cities[df_cities.city.isin(liste_warehouses_1)]
df_warehouse_1.index = np.arange(0,len(df_warehouse_1))

In [18]:
df_cities_1["warehouse"] = df_cities_1.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_1))
df_cities_1["index_warehouse"] = df_cities_1.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_1,return_index=True))
df_warehouse_1['index_warehouse'] = df_warehouse_1.index.copy()

d:\Documents\Cours\3A\M2AD\Novembre\edc_ekimetrics\venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [19]:
m = folium.Map()

df_cities_1.apply(lambda point : maps.plotDot(m,point,color=maps.colors[point.index_warehouse],with_tooltip=False), axis = 1)

#Set the zoom to the maximum possible
m.fit_bounds(m.get_bounds())

df_warehouse_1.apply(lambda point : maps.plotMarker(m,point,color = maps.colors[int(point.index_warehouse)]), axis = 1)

m

In [20]:
df_routes_separate_1_small = processing.try_scenario(df_cities_1,df_routes=df_routes.sample(n_trajets,random_state=42))
int(df_routes_separate_1_small.total_distance_f.sum()* CO2_per_km)

57643

In [21]:
df_routes_separate_optimized_1_small = processing.try_scenario(df_cities_1,df_routes=df_routes.sample(n_trajets,random_state=42),with_optim=True)
int(df_routes_separate_optimized_1_small.total_distance_f.sum()* CO2_per_km)

52408

## Scénario 2

On déplace 1 entrepôt : celui de Clermont-Ferrant à Rennes

In [22]:
df_cities_2 = pd.read_csv("../data/cities.csv")
liste_warehouses_2 = ['Cergy','Reims','Avignon','Montauban','Rennes']

In [23]:
df_warehouse_2 = df_cities[df_cities.city.isin(liste_warehouses_2)]
df_warehouse_2.index = np.arange(0,len(df_warehouse_2))

df_cities_2["warehouse"] = df_cities_2.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_2))
df_cities_2["index_warehouse"] = df_cities_2.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_2,return_index=True))
df_warehouse_2['index_warehouse'] = df_warehouse_2.index.copy()

d:\Documents\Cours\3A\M2AD\Novembre\edc_ekimetrics\venv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
df_cities_2.loc[df_cities_2.city=='Perpignan', 'warehouse'] = 'Avignon'
df_cities_2.loc[df_cities_2.city=='Perpignan', 'index_warehouse'] = 2

In [25]:
m = folium.Map()

df_cities_2.apply(lambda point : maps.plotDot(m,point,color=maps.colors[point.index_warehouse],with_tooltip=False), axis = 1)
df_warehouse_2.apply(lambda point : maps.plotMarker(m,point,color = maps.colors[int(point.index_warehouse)]), axis = 1)

m.fit_bounds(m.get_bounds())
m

In [26]:
df_routes_separate_2_small = processing.try_scenario(df_cities_2,df_routes=df_routes.sample(n_trajets,random_state=42))
int(df_routes_separate_2_small.total_distance_f.sum()* CO2_per_km)

52078

In [27]:
df_routes_separate_optimized_2_small = processing.try_scenario(df_cities_2,df_routes=df_routes.sample(n_trajets,random_state=42),with_optim=True)
int(df_routes_separate_optimized_2_small.total_distance_f.sum()* CO2_per_km)

49438

## Scénario 3

On déplace 2 entrepôts : celui de Clermont-Ferrant à Rennes, et celui de Reims à Mulhouse

In [28]:
df_cities_3 = pd.read_csv("../data/cities.csv")

liste_warehouses_3 = ['Cergy','Mulhouse','Avignon','Montauban','Rennes']
df_warehouse_3 = df_cities[df_cities.city.isin(liste_warehouses_3)]
df_warehouse_3.index = np.arange(0,len(df_warehouse_3))
df_warehouse_3['index_warehouse'] = df_warehouse_3.index 

df_cities_3["warehouse"] = df_cities_3.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_3))
df_cities_3["index_warehouse"] = df_cities_3.city.apply(lambda row : processing.find_closest_warehouse(row,liste_warehouses_3,return_index=True))

d:\Documents\Cours\3A\M2AD\Novembre\edc_ekimetrics\venv\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
df_cities_3.loc[df_cities_3.city=='Annecy', 'warehouse'] = 'Avignon'
df_cities_3.loc[df_cities_3.city=='Annecy', 'index_warehouse'] = 2

df_cities_3.loc[df_cities_3.city=='Perpignan', 'warehouse'] = 'Avignon'
df_cities_3.loc[df_cities_3.city=='Perpignan', 'index_warehouse'] = 2

In [30]:
m = folium.Map()

df_cities_3.apply(lambda point : maps.plotDot(m,point,color=maps.colors[point.index_warehouse],with_tooltip=False), axis = 1)
df_warehouse_3.apply(lambda point : maps.plotMarker(m,point,color = maps.colors[int(point.index_warehouse)]), axis = 1)

m.fit_bounds(m.get_bounds())
m

In [31]:
df_routes_separate_3_small = processing.try_scenario(df_cities_3,df_routes=df_routes.sample(n_trajets,random_state=42))
int(df_routes_separate_3_small.total_distance_f.sum()* CO2_per_km)

51612

In [32]:
df_routes_separate_optimized_3_small = processing.try_scenario(df_cities_3,df_routes=df_routes.sample(n_trajets,random_state=42),with_optim=True)
int(df_routes_separate_optimized_3_small.total_distance_f.sum()* CO2_per_km)

49820